In [0]:
''''
Author : Rutvik Parekh
Objective: Extracting divorce rates for couples with same and different political party affiliations 


'''


# Take the data from the 20160930 election month and store it in a dataframe.

import pandas as pd
import pdb
import os
from google.colab import drive

from datetime import datetime
drive.mount('/content/drive')


#Read all files and merge into 1 ( All counties)

dataframes = []

counter = 0
total = len(os.listdir('/content/drive/My Drive/5Counties_2016')) 
for filename in os.listdir('/content/drive/My Drive/5Counties_2016'):
  data = pd.read_csv('/content/drive/My Drive/5Counties_2016/'+filename, sep="\t", header=None,low_memory=False)     
  data.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension","Email address"]
  data =  data.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]         
  data = data.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])      
  dataframes.append(data)

print("Processed {} of {} files".format(counter,total))
print('\n')

new_dataframe = pd.DataFrame()
temp_dataframe = pd.DataFrame()

#for i,df in enumerate(dataframes) :
temp_dataframe = pd.concat(dataframes)
temp_dataframe = temp_dataframe[temp_dataframe['Party Affiliation'].isin(['DEM', 'REP'])]
print(temp_dataframe.shape)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processed 0 of 5 files


(543150, 14)


In [0]:
# Separate the couples who have same and different party affiliations from 20160930 election data.

#print (temp_dataframe.head(5))
pd.options.display.max_columns = None
#temp=temp_dataframe.sample(n=20000)
temp2 = temp_dataframe
#temp2=temp_dataframe
#print(temp.head(5))
#print(temp['Residence Address Line 2'].head(5))
temp2['All'] = temp2['Residence Address Line 1'].str.cat(temp2[['Residence Address Line 2']], sep=' - ')
#print(temp['All'].head(5))
df=temp2.groupby('All')
data=pd.DataFrame()

df1 = pd.DataFrame()
df2 = pd.DataFrame()
couples1 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male", "Voter_ID_Female", "Name_First_Female", "Name_Last_Female"])
couples2 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male", "Voter_ID_Female", "Name_First_Female", "Name_Last_Female"])
counter = 0
print(temp2.shape)
for name,group in df:

    if counter%30000 == 0 :
      print("Processed {} of items".format(counter) )
    counter += 1

    if group.shape[0]==2 and group[group.Gender == 'F'].shape[0] == 1 and group[group.Gender == 'M'].shape[0] == 1:
        group_name=group.groupby('Name Last')
        for name1,group1 in group_name:
            if group1.shape[0]==2:
                birthdate1 = pd.to_datetime(group1.iloc[0]['Birth Date'])
                birthdate2 = pd.to_datetime(group1.iloc[1]['Birth Date'])
                birthdateDiff = abs(birthdate1 - birthdate2)
                #birthdateDiffInt = birthdateDiff.astype(int)
                fifteenYears = pd.Timedelta(15, 'Y')
                if birthdateDiff < fifteenYears:
                  #print(name1)
                  #print(group1)
                  data=data.append([group])
                  if (group1.iloc[0,13] == group1.iloc[1,13]) :
                    df1 = df1.append(group)
                    couples1AllColumnsMale = pd.DataFrame()
                    couples1AllColumnsFemale = pd.DataFrame()
                    #display(group1.head())
                    #display(group1.head())
                    tempCouples1 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"])
                    couples1AllColumnsMale = group1[group1.Gender == 'M']
                    #display(group1.take([0]))
                    #display(couples1AllColumnsFemale.head())
                    couples1AllColumnsFemale = group1[group1.Gender == 'F']
                    tempCouples1[["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"]] = couples1AllColumnsMale[['Voter ID', 'Name First', 'Name Last']]
                    #tempCouples1[['Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female']] = couples1AllColumnsFemale[['Voter ID', 'Name First', 'Name Last']]
                    tempCouples1['Voter_ID_Female'] = couples1AllColumnsFemale.to_numpy()[0][0]
                    tempCouples1['Name_First_Female'] = couples1AllColumnsFemale.to_numpy()[0][2]
                    tempCouples1[ 'Name_Last_Female'] = couples1AllColumnsFemale.to_numpy()[0][1]
                    #display(tempCouples1.head())
                    couples1 = couples1.append(tempCouples1)
                  else :
                    df2 = df2.append(group)
                    couples2AllColumnsMale = pd.DataFrame()
                    couples2AllColumnsFemale = pd.DataFrame()
                    #display(group1.head())
                    #display(group1.head())
                    tempCouples2 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"])
                    couples2AllColumnsMale = group1[group1.Gender == 'M']
                    couples2AllColumnsFemale = group1[group1.Gender == 'F']
                    tempCouples2[["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"]] = couples2AllColumnsMale[['Voter ID', 'Name First', 'Name Last']]
                    #tempCouples1[['Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female']] = couples1AllColumnsFemale[['Voter ID', 'Name First', 'Name Last']]
                    tempCouples2['Voter_ID_Female'] = couples2AllColumnsFemale.to_numpy()[0][0]
                    tempCouples2['Name_First_Female'] = couples2AllColumnsFemale.to_numpy()[0][2]
                    tempCouples2[ 'Name_Last_Female'] = couples2AllColumnsFemale.to_numpy()[0][1]
                    #display(tempCouples1.head())
                    couples2 = couples2.append(tempCouples2)
print('Processing Complete')



(543150, 15)
Processed 0 of items


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: FutureWarning: M and Y units are deprecated and will be removed in a future version.


Processed 30000 of items
Processed 60000 of items
Processed 90000 of items
Processed 120000 of items
Processed 150000 of items
Processed 180000 of items
Processed 210000 of items
Processed 240000 of items
Processed 270000 of items
Processed 300000 of items
Processed 330000 of items
Processing Complete


In [0]:
print(couples1.shape)
display(couples1.head())

(10228, 6)


,Voter_ID_Male,Name_First_Male,Name_Last_Male,Voter_ID_Female,Name_First_Female,Name_Last_Female
134309,118607508,Craig,Heitzler,NaN,NaN,NaN
111642,116176198,Cynthia,Blaich,NaN,NaN,NaN
18451,100412797,Cynthia,Curtis,NaN,NaN,NaN
65165,100553932,Lou,Liguori,NaN,NaN,NaN
891,100383315,Christian,Hart,NaN,NaN,NaN


In [0]:
# Take the data from the 20170630 election month and store it in a dataframe.

import pandas as pd
import pdb
import os
from google.colab import drive

from datetime import datetime
drive.mount('/content/drive')


#Read all files and merge into 1 ( All counties)

dataframes_2017 = []
total_2017 = len(os.listdir('/content/drive/My Drive/5Counties_2017/')) 
for filename in os.listdir('/content/drive/My Drive/5Counties_2017'):
  data_2017 = pd.read_csv('/content/drive/My Drive/5Counties_2017/'+filename, sep="\t", header=None,low_memory=False)        
  data_2017.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension","Email address"]
  data_2017 =  data_2017.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]        
  data_2017 = data_2017.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])
  dataframes_2017.append(data_2017)
  print('\n')
temp_dataframe_2017 = pd.DataFrame()

#for i,df in enumerate(dataframes) :
temp_dataframe_2017 = pd.concat(dataframes_2017)
temp_dataframe_2017 = temp_dataframe_2017[temp_dataframe_2017['Party Affiliation'].isin(['DEM', 'REP'])]
print(temp_dataframe_2017.shape)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).










(531001, 14)


In [0]:
#print (temp_dataframe.head(5))
pd.options.display.max_columns = None
#temp=temp_dataframe.sample(n=20000)
temp2_2017 = temp_dataframe_2017
#pdb.set_trace()
#temp2=temp_dataframe
#print(temp.head(5))
#print(temp['Residence Address Line 2'].head(5))
temp2_2017['All'] = temp2_2017['Residence Address Line 1'].str.cat(temp2_2017[['Residence Address Line 2']], sep=' - ')
#print(temp['All'].head(5))
df_2017=temp2_2017.groupby('All')
data_2017=pd.DataFrame()

df1_2017 = pd.DataFrame()
df2_2017 = pd.DataFrame()
couples12017 = pd.DataFrame(columns=['Voter_ID_Male', 'Name_First_Male', 'Name_Last_Male', 'Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female'])
couples22017 = pd.DataFrame(columns=['Voter_ID_Male', 'Name_First_Male', 'Name_Last_Male', 'Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female'])
counter = 0
print(temp2_2017.shape)
#pdb.set_trace()
for name,group in df_2017:
    if counter%30000 == 0 :
      print("Processed {} of items".format(counter) )
    counter += 1

    if group.shape[0]==2 and group[group.Gender == 'F'].shape[0] == 1 and group[group.Gender == 'M'].shape[0] == 1:
        #pdb.set_trace()
        group_name=group.groupby('Name Last')
        for name1,group1 in group_name:
            if group1.shape[0]==2:
                birthdate1 = pd.to_datetime(group1.iloc[0]['Birth Date'])
                birthdate2 = pd.to_datetime(group1.iloc[1]['Birth Date'])
                birthdateDiff = abs(birthdate1 - birthdate2)
                #birthdateDiffInt = birthdateDiff.astype(int)
                fifteenYears = pd.Timedelta(15, 'Y')
                if birthdateDiff < fifteenYears:
                  data_2017=data_2017.append([group])
                  if (group1.iloc[0,13] == group1.iloc[1,13]) :
                    df1_2017 = df1_2017.append(group)
                    couples12017AllColumnsMale = pd.DataFrame()
                    couples12017AllColumnsFemale = pd.DataFrame()
                    #display(group1.head())
                    #display(group1.head())
                    tempCouples12017 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"])
                    couples12017AllColumnsMale = group1[group1.Gender == 'M']
                    #display(group1.take([0]))
                    #display(couples1AllColumnsFemale.head())
                    couples12017AllColumnsFemale = group1[group1.Gender == 'F']
                    tempCouples12017[["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"]] = couples12017AllColumnsMale[['Voter ID', 'Name First', 'Name Last']]
                    #tempCouples1[['Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female']] = couples1AllColumnsFemale[['Voter ID', 'Name First', 'Name Last']]
                    tempCouples12017['Voter_ID_Female'] = couples12017AllColumnsFemale.to_numpy()[0][0]
                    tempCouples12017['Name_First_Female'] = couples12017AllColumnsFemale.to_numpy()[0][2]
                    tempCouples12017[ 'Name_Last_Female'] = couples12017AllColumnsFemale.to_numpy()[0][1]
                    #display(tempCouples1.head())
                    couples12017 = couples12017.append(tempCouples12017)
                  else :
                    df2_2017 = df2_2017.append(group)
                    couples22017AllColumnsMale = pd.DataFrame()
                    couples22017AllColumnsFemale = pd.DataFrame()
                    #display(group1.head())
                    #display(group1.head())
                    tempCouples22017 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"])
                    couples22017AllColumnsMale = group1[group1.Gender == 'M']
                    #display(group1.take([0]))
                    #display(couples1AllColumnsFemale.head())
                    couples22017AllColumnsFemale = group1[group1.Gender == 'F']
                    tempCouples22017[["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"]] = couples22017AllColumnsMale[['Voter ID', 'Name First', 'Name Last']]
                    #tempCouples1[['Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female']] = couples1AllColumnsFemale[['Voter ID', 'Name First', 'Name Last']]
                    tempCouples22017['Voter_ID_Female'] = couples22017AllColumnsFemale.to_numpy()[0][0]
                    tempCouples22017['Name_First_Female'] = couples22017AllColumnsFemale.to_numpy()[0][2]
                    tempCouples22017[ 'Name_Last_Female'] = couples22017AllColumnsFemale.to_numpy()[0][1]
                    #display(tempCouples1.head())
                    couples22017 = couples22017.append(tempCouples22017)
                    #pdb.set_trace()

(531001, 15)
Processed 0 of items


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: FutureWarning: M and Y units are deprecated and will be removed in a future version.


Processed 30000 of items
Processed 60000 of items
Processed 90000 of items
Processed 120000 of items
Processed 150000 of items
Processed 180000 of items
Processed 210000 of items
Processed 240000 of items
Processed 270000 of items
Processed 300000 of items
Processed 330000 of items


In [0]:
print(df1.shape)
print(df2.shape)
print(df1_2017.shape)
print(df2_2017.shape)

(105810, 15)
(13550, 15)
(0, 0)
(0, 0)


In [0]:
display(couples1.shape)
display(couples2.shape)
display(couples12017.shape)
display(couples22017.shape)

display(couples22017.head(20))

(9214, 6)

(1355, 6)

(9050, 6)

(1319, 6)

,Voter_ID_Male,Name_First_Male,Name_Last_Male,Voter_ID_Female,Name_First_Female,Name_Last_Female
151292,111702936,Garrett,Mullee,111706826,Nancy,Mullee
80259,123410555,Andrew,Fischer,123410510,Mary,Fischer
170192,105403407,Larry,Goodling,105403408,Dorothy,Goodling
20031,116265108,Patrick,Noyes,106242964,Laura,Noyes
102471,100415505,Holton,Howell,100411230,Teresa,Howell
68578,100393946,David,Raulerson,100412036,Linell,Raulerson
55496,100559324,Chad,Griffin,100535487,Rachel,Griffin
19509,104755524,John,Brigan,100468048,Maritza,Brigan
110473,100396917,Peter,Ward,100396923,Susan,Ward
97177,100412180,William,Pacheco,100412723,Alicia,Pacheco


In [0]:
import numpy as np
couples_samePPA_merged = pd.merge(couples1, couples12017, on=['Voter_ID_Male','Voter_ID_Female'], how='left', indicator='Exist')
#couples_samePPA_merged.drop(['Name_First_Male','Name_Last_Male','Name_First_Female','Name_Last_Female'], inplace=True, axis=1)
display(couples_samePPA_merged.head())
couples_samePPA_merged[['Exist']] = np.where(couples_samePPA_merged.Exist == 'both', True, False)
display(couples_samePPA_merged['Exist'].value_counts())

samePPA_valueCounts = couples_samePPA_merged['Exist'].value_counts().tolist()
sameDivorceRate = samePPA_valueCounts[1]/(samePPA_valueCounts[0]+samePPA_valueCounts[1])
couples_diffPPA_merged = pd.merge(couples2, couples22017, on=['Voter_ID_Male','Voter_ID_Female'], how='left', indicator='Exist')
#couples_diffPPA_merged.drop(['Name_First_Male','Name_Last_Male','Name_First_Female','Name_Last_Female'], inplace=True, axis=1)
couples_diffPPA_merged['Exist'] = np.where(couples_diffPPA_merged.Exist == 'both', True, False)

display(couples_diffPPA_merged['Exist'].value_counts())

diffPPA_valueCounts = couples_diffPPA_merged['Exist'].value_counts().tolist()
diffDivorceRate = diffPPA_valueCounts[1]/(diffPPA_valueCounts[0]+diffPPA_valueCounts[1])

,Voter_ID_Male,Name_First_Male_x,Name_Last_Male_x,Voter_ID_Female,Name_First_Female_x,Name_Last_Female_x,Name_First_Male_y,Name_Last_Male_y,Name_First_Female_y,Name_Last_Female_y,Exist
0,118607508,Craig,Heitzler,119437179,Melissa,Heitzler,Craig,Heitzler,Melissa,Heitzler,both
1,116313828,Michael,Blaich,116176198,Cynthia,Blaich,NaN,NaN,NaN,NaN,left_only
2,100469337,John,Curtis,100412797,Cynthia,Curtis,John,Curtis,Cynthia,Curtis,both
3,103749702,Donald,Liguori,100553932,Lou,Liguori,Donald,Liguori,Lou,Liguori,both
4,100383315,Christian,Hart,100423402,Patricia,Hart,Christian,Hart,Patricia,Hart,both


True     8276
False     938
Name: Exist, dtype: int64

True     1192
False     163
Name: Exist, dtype: int64

In [0]:
print("Same political party affiliation divorce rate:")
display(sameDivorceRate*100)
print("Different political party affiliation divorce rate:")
display(diffDivorceRate*100)

Same political party affiliation divorce rate:


10.180160625135663

Different political party affiliation divorce rate:


12.029520295202952